# IRIS SSP AUS maps


This example uses IRIS to plot maps of ensemble anomalies for SSP585 future senario.

In this notebook we demonstrate:

- load ensemble data  
- add ensemble number into data corrdinate
- plot ensemble anomaly

The following material uses Coupled Model Intercomparison Project (CMIP6) collections. The CMIP6 terms of use are found[here](https://pcmdi.llnl.gov/CMIP6/TermsOfUse/TermsOfUse6-1.html). For more information on the collection, please click [here]( https://geonetwork.nci.org.au/geonetwork/srv/eng/catalog.search#/metadata/f6600_2266_8675_3563).

---


- Authors: NCI Virtual Research Environment team
- Keywords: CMIP6, IRIS
- Software: Python, IRIS
- Create Date: 2020-Sep
---

In [1]:
import iris

In [2]:
!ls /g/data/oi10/replicas/CMIP6/ScenarioMIP/CNRM-CERFACS/CNRM-CM6-1/ssp585/

r1i1p1f2  r2i1p1f2  r3i1p1f2  r4i1p1f2	r5i1p1f2  r6i1p1f2


### Load multipul datasets

In [3]:
filenames=[]
for i in range(1,7):
    Dir='/g/data/oi10/replicas/CMIP6/ScenarioMIP/CNRM-CERFACS/CNRM-CM6-1/ssp585/r'+str(i)+'i1p1f2/Amon/tas/gr/*/'
    fname='tas_Amon_CNRM-CM6-1_ssp585_r'+str(i)+'i1p1f2_gr_201501-210012.nc'
    file=Dir+fname
    filenames.append(file)
cubes = iris.load(filenames, 'air_temperature')
cubes

/g/data/dk92/apps/anaconda3/2020.12/envs/NCI-data-analysis/lib/python3.7/site-packages/iris/fileformats/cf.py:803: UserWarning: Missing CF-netCDF measure variable 'areacella', referenced by netCDF variable 'tas'
  warnings.warn(message % (variable_name, nc_var_name))
/g/data/dk92/apps/anaconda3/2020.12/envs/NCI-data-analysis/lib/python3.7/site-packages/iris/fileformats/cf.py:803: UserWarning: Missing CF-netCDF measure variable 'areacella', referenced by netCDF variable 'tas'
  warnings.warn(message % (variable_name, nc_var_name))
/g/data/dk92/apps/anaconda3/2020.12/envs/NCI-data-analysis/lib/python3.7/site-packages/iris/fileformats/cf.py:803: UserWarning: Missing CF-netCDF measure variable 'areacella', referenced by netCDF variable 'tas'
  warnings.warn(message % (variable_name, nc_var_name))
/g/data/dk92/apps/anaconda3/2020.12/envs/NCI-data-analysis/lib/python3.7/site-packages/iris/fileformats/cf.py:803: UserWarning: Missing CF-netCDF measure variable 'areacella', referenced by netCDF

[<iris 'Cube' of air_temperature / (K) (time: 1032; latitude: 128; longitude: 256)>,
<iris 'Cube' of air_temperature / (K) (time: 1032; latitude: 128; longitude: 256)>,
<iris 'Cube' of air_temperature / (K) (time: 1032; latitude: 128; longitude: 256)>,
<iris 'Cube' of air_temperature / (K) (time: 1032; latitude: 128; longitude: 256)>,
<iris 'Cube' of air_temperature / (K) (time: 1032; latitude: 128; longitude: 256)>,
<iris 'Cube' of air_temperature / (K) (time: 1032; latitude: 128; longitude: 256)>]

### Choose sub region --Australia

In [4]:
region = cubes.extract(iris.Constraint(latitude=lambda v: -50 < v < -10, longitude=lambda vv: 110 < vv < 155))

In [5]:
region

[<iris 'Cube' of air_temperature / (K) (time: 1032; latitude: 29; longitude: 32)>,
<iris 'Cube' of air_temperature / (K) (time: 1032; latitude: 29; longitude: 32)>,
<iris 'Cube' of air_temperature / (K) (time: 1032; latitude: 29; longitude: 32)>,
<iris 'Cube' of air_temperature / (K) (time: 1032; latitude: 29; longitude: 32)>,
<iris 'Cube' of air_temperature / (K) (time: 1032; latitude: 29; longitude: 32)>,
<iris 'Cube' of air_temperature / (K) (time: 1032; latitude: 29; longitude: 32)>]

###  Adding realization coordinate

In [6]:
def realization_metadata(cube, field, fname):
    """
    A function which modifies the cube's metadata to add a "realization"
    (ensemble member) coordinate from the filename if one doesn't already exist
    in the cube.

    """
    # add an ensemble member coordinate if one doesn't already exist
    if not cube.coords('realization'):
        # the ensemble member is encoded in the filename as *_???.nc where ???
        # is the ensemble member
        realization_number = fname[-27]

        import iris.coords
        realization_coord = iris.coords.AuxCoord(np.int32(realization_number),
                                                 'realization')
        cube.add_aux_coord(realization_coord)

In [7]:
import numpy as np
for i in range(6): 
    realization_metadata(region[i], 'air_temperature', filenames[i])
cubes

[<iris 'Cube' of air_temperature / (K) (time: 1032; latitude: 128; longitude: 256)>,
<iris 'Cube' of air_temperature / (K) (time: 1032; latitude: 128; longitude: 256)>,
<iris 'Cube' of air_temperature / (K) (time: 1032; latitude: 128; longitude: 256)>,
<iris 'Cube' of air_temperature / (K) (time: 1032; latitude: 128; longitude: 256)>,
<iris 'Cube' of air_temperature / (K) (time: 1032; latitude: 128; longitude: 256)>,
<iris 'Cube' of air_temperature / (K) (time: 1032; latitude: 128; longitude: 256)>]

### Plotting

Note: It is better to run the next cell on VDI as compute notes do not have internet access. 

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import iris.plot as iplt
import matplotlib
%matplotlib inline

# Create a wider than normal figure to support our many plots
plt.figure(figsize=(12, 8), dpi=100)

    # Also manually adjust the spacings which are used when creating subplots
plt.gcf().subplots_adjust(hspace=0.05, wspace=0.05, top=0.95, bottom=0.05,
                              left=0.075, right=0.925)

    # calculate ensemble average
avg=(region[0].collapsed('time', iris.analysis.MEAN)  + region[1].collapsed('time', iris.analysis.MEAN) 
   + region[2].collapsed('time', iris.analysis.MEAN)+ region[3].collapsed('time', iris.analysis.MEAN) 
   + region[4].collapsed('time', iris.analysis.MEAN) + region[5].collapsed('time', iris.analysis.MEAN))/6

#loop over all realizations
for i in range(6):
        cube = region[i].collapsed('time', iris.analysis.MEAN) - avg

  
        plt.subplot(2, 3, i+1)

        cf = iplt.contourf(cube, 20)
        
    # add coastlines
        plt.gca().coastlines()
        colorbar = plt.colorbar(cf, orientation='horizontal')
    # make an axes to put the shared colorbar in
        #colorbar_axes = plt.gcf().add_axes([0.35, 0.01, 0.3, 0.05])
        #colorbar = plt.colorbar(cf, colorbar_axes, orientation='horizontal')

        #colorbar.locator = matplotlib.ticker.MaxNLocator(8)
        #colorbar.update_ticks()
        plt.title('r'+str(i+1)+'i1p1f2')

plt.suptitle('Air temperature ensemble anomaly', y=1)

iplt.show()
